In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import models,transforms
import matplotlib.pyplot as plt
import pickle
from collections import OrderedDict
import csv
import collections
from  PIL import Image
from tqdm.notebook import tqdm_notebook
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')
import math
device = torch.device("mps" if torch.has_mps else "cpu")

In [2]:
### load data 
# create df to contain all identities, their image file names, their ethnicities
path = "data/RFW/images/test/txts/"
img_path = 'data/RFW/images/test/data/'

# African images
african_images = pd.read_csv(path + 'African/African_images.txt', sep="\t", header=None)
african_images.columns = ['File', 'Label']
african_images['identityID'] = african_images['File'].str[:-9]
african_images['faceID'] = african_images['File'].str[-8:-4]
african_images['Ethnicity'] = 'African'
# Asian images
asian_images = pd.read_csv(path + 'Asian/Asian_images.txt', sep="\t", header=None)
asian_images.columns = ['File', 'Label']
asian_images['identityID'] = asian_images['File'].str[:-9]
asian_images['faceID'] = asian_images['File'].str[-8:-4]
asian_images['Ethnicity'] = 'Asian'
# Caucasian images
caucasian_images = pd.read_csv(path + 'Caucasian/Caucasian_images.txt', sep="\t", header=None)
caucasian_images.columns = ['File', 'Label']
caucasian_images['identityID'] = caucasian_images['File'].str[:-9]
caucasian_images['faceID'] = caucasian_images['File'].str[-8:-4]
caucasian_images['Ethnicity'] = 'Caucasian'
# Indian images
indian_images = pd.read_csv(path + 'Indian/Indian_images.txt', sep="\t", header=None)
indian_images.columns = ['File', 'Label']
indian_images['identityID'] = indian_images['File'].str[:-9]
indian_images['faceID'] = indian_images['File'].str[-8:-4]
indian_images['Ethnicity'] = 'Indian'
all_images = pd.concat([african_images,asian_images,caucasian_images,indian_images])

# remove any duplicate identities
v = all_images.reset_index().groupby('identityID').Ethnicity.nunique()
dup = v[v>1].index.tolist()
all_images = all_images[~all_images['identityID'].isin(dup)]

# get first image from each identity and use it as reference
identities = np.array(all_images.identityID.unique().tolist()).astype(object)
file_end =  np.array('_0001.jpg'.split()*len(identities)).astype(object)
first_images = identities + file_end

references = all_images[all_images['File'].isin(first_images)]
candidates = all_images[~all_images['File'].isin(first_images)]

In [3]:
### ResNet50 model architecture
__all__ = ['ResNet', 'resnet50']

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=8631, include_top=True):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.include_top = include_top
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=True)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        
        if not self.include_top:
            return x
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

In [4]:
### load model, assign weights, send to device
model_ft = resnet50()
fname = 'weights/resnet50_ft_weight.pkl'

with open(fname, 'rb') as f:
    weights = pickle.load(f, encoding='latin1')

own_state = model_ft.state_dict()
for name, param in weights.items():
    if name in own_state:
        try:
            own_state[name].copy_(torch.from_numpy(param))
        except Exception:
            raise RuntimeError('While copying the parameter named {}, whose dimensions in the model are {} and whose '\
                                'dimensions in the checkpoint are {}.'.format(name, own_state[name].size(), param.shape))
    else:
        raise KeyError('unexpected key "{}" in state_dict'.format(name))
model_ft = torch.nn.Sequential(*(list(model_ft.children())[:-1]))
model_ft.to(device)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 

In [5]:
# create dataset class for RFW
class resnetRFW(data.Dataset):
    
    '''
    This will be a class to load data from RFW for resnet50 model
    '''
     
    mean_bgr = np.array([91.4953, 103.8827, 131.0912])  # from resnet50_ft.prototxt

    def __init__(self,img_path,img_df):
        """
        :param img_path: dataset directory
        :param img_df: contains image file names and other information
        """
        assert os.path.exists(img_path), "root: {} not found.".format(img_path)
        self.img_path = img_path
        self.img_df = img_df
        self.img_info = []

        for i, row in self.img_df.iterrows():
            self.img_info.append({
                'img_file': row.Ethnicity + '/' + row.identityID + '/' + row.File,
                'identityID': row.identityID,
                'Ethnicity': row.Ethnicity,
                'faceID': row.faceID,
            })
            if i % 5000 == 0:
                print("processing: {} images".format(i))

    def __len__(self):
        return len(self.img_info)

    def __getitem__(self, index):
        info = self.img_info[index]
        img_file = info['img_file']
        img = Image.open(os.path.join(self.img_path, img_file))
        img = transforms.Resize(256)(img)
        img = transforms.CenterCrop(224)(img)
        img = np.array(img, dtype=np.uint8)
        assert len(img.shape) == 3  # assumes color images and no alpha channel

        Ethnicity = info['Ethnicity']
        identityID = info['identityID']
        faceID = info['faceID']
        return self.transform(img), identityID, Ethnicity, faceID
  

    def transform(self, img):
        img = img[:, :, ::-1]  # RGB -> BGR
        img = img.astype(np.float32)
        img -= self.mean_bgr
        img = img.transpose(2, 0, 1)  # C x H x W
        img = torch.from_numpy(img).float()
        return img

    def untransform(self, img, lbl):
        img = img.numpy()
        img = img.transpose(1, 2, 0)
        #img += self.mean_bgr
        img = img.astype(np.uint8)
        img = img[:, :, ::-1]
        return img, lbl


kwargs = {'num_workers': 4, 'pin_memory': True} if torch.cuda.is_available() else {}
# load reference images
reference_dataset = resnetRFW(img_path,references.reset_index(drop=True))
reference_loader = torch.utils.data.DataLoader(reference_dataset, batch_size=4, shuffle=False, **kwargs)
# load candidate images
candidate_dataset = resnetRFW(img_path,candidates.reset_index(drop=True))
candidate_loader = torch.utils.data.DataLoader(candidate_dataset, batch_size=4, shuffle=False, **kwargs)

processing: 0 images
processing: 5000 images
processing: 10000 images
processing: 0 images
processing: 5000 images
processing: 10000 images
processing: 15000 images
processing: 20000 images
processing: 25000 images


In [6]:
def apply_model(model,dataloader,file_prefix,device):
    model.eval()
    outputs = []
    identities = []
    ethnicities = []
    faceIDs = []
    with torch.no_grad():
        for i, (imgs, identityID, Ethnicity, faceID) in tqdm_notebook(enumerate(dataloader),total=len(dataloader)):
            imgs = imgs.to(device)
            x = model(imgs)
            out = x.view(x.size(0),-1)
            outputs.append(out)
            identities.append(np.array(identityID))
            ethnicities.append(np.array(Ethnicity))
            faceIDs.append(np.array(faceID))

    outputs=torch.cat(outputs)
    identities= np.concatenate(np.array(identities)).ravel()
    ethnicities= np.concatenate(np.array(ethnicities)).ravel()
    faceIDs= np.concatenate(np.array(faceIDs)).ravel()

    torch.save(outputs, file_prefix + '_outputs.pt')
    np.save(file_prefix + '_identities.npy', identities)
    np.save(file_prefix + '_ethnicities.npy', ethnicities)
    np.save(file_prefix + '_faceIDs.npy', faceIDs)
    return outputs, identities, ethnicities, faceIDs

In [7]:
reference_outputs, reference_identities, reference_ethnicities, reference_faceIDs = apply_model(model_ft,reference_loader,'outputs/RFW/ft/reference2',device)

  0%|          | 0/2851 [00:00<?, ?it/s]

In [8]:
candidate_outputs, candidate_identities, candidate_ethnicities, candidate_faceIDs = apply_model(model_ft,candidate_loader,'outputs/RFW/ft/candidate2',device)

  0%|          | 0/7280 [00:00<?, ?it/s]

In [9]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return torch.matmul(A_mA, B_mB.T) / torch.sqrt(torch.matmul(ssA[:, None],ssB[None]))
def cos_sim(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

cor = corr2_coeff(reference_outputs,candidate_outputs).cpu().detach().numpy()
cos = cos_sim(reference_outputs,candidate_outputs).cpu().detach().numpy()

In [10]:
# Face Identification

'''
here i want to do face identification
basically take the outputs from the candidates and check against all references
take the reference image that has highest correlation/similarity
if the identities match then correct otherwise wrong
'''

cor_identification = pd.DataFrame(columns=['candidate_identity','candidate_ethnicity','reference_identity'])
for i, cor_row in tqdm_notebook(enumerate(cor.T),total=len(cor.T)):
    identity = candidate_identities[i]
    ethnicity = candidate_ethnicities[i]
    max_ref = np.argmax(cor_row)
    reference_identity = reference_identities[max_ref]
    match = 1 if identity == reference_identity else  0
    row = {'candidate_identity': identity,
            'candidate_ethnicity': ethnicity, 
            'reference_identity': reference_identity,
            'match': match}

    cor_identification = cor_identification.append(row,ignore_index=True)

cos_identification = pd.DataFrame(columns=['candidate_identity','candidate_ethnicity','reference_identity'])
for i, cos_row in tqdm_notebook(enumerate(cos.T),total=len(cos.T)):
    identity = candidate_identities[i]
    ethnicity = candidate_ethnicities[i]
    max_ref = np.argmax(cos_row)
    reference_identity = reference_identities[max_ref]
    match = 1 if identity == reference_identity else  0
    row = {'candidate_identity': identity,
            'candidate_ethnicity': ethnicity, 
            'reference_identity': reference_identity,
            'match': match}

    cos_identification = cos_identification.append(row,ignore_index=True)

cos_id_acc = pd.DataFrame(columns=['ethnicity','accuracy'])
for ethnicity in cos_identification.candidate_ethnicity.unique():
    eth_cos = cos_identification.loc[cos_identification.candidate_ethnicity == ethnicity]
    accuracy = sum(eth_cos.match)/len(eth_cos)
    #print(len(eth_cos))
    row = {'ethnicity': ethnicity,
            'accuracy': accuracy}

    cos_id_acc = cos_id_acc.append(row,ignore_index=True)
print(cos_id_acc)

cor_id_acc = pd.DataFrame(columns=['ethnicity','accuracy'])
for ethnicity in cor_identification.candidate_ethnicity.unique():
    eth_cor = cor_identification.loc[cor_identification.candidate_ethnicity == ethnicity]
    accuracy = sum(eth_cor.match)/len(eth_cor)
    #print(len(eth_cos))
    row = {'ethnicity': ethnicity,
            'accuracy': accuracy}

    cor_id_acc = cor_id_acc.append(row,ignore_index=True)
print(cor_id_acc)

  0%|          | 0/29117 [00:00<?, ?it/s]

  0%|          | 0/29117 [00:00<?, ?it/s]

   ethnicity  accuracy
0    African  0.496348
1      Asian  0.530124
2  Caucasian  0.633877
3     Indian  0.625377
   ethnicity  accuracy
0    African  0.494455
1      Asian  0.529428
2  Caucasian  0.632771
3     Indian  0.623184


In [11]:
### face verification
thresh = 0.65
cos_verification = pd.DataFrame(columns=['reference_identity','reference_ethnicity','TP','TN','FP','FN','matches','not_matches'])
for i, cos_row in tqdm_notebook(enumerate(cos),total=len(cos)):
  identity = reference_identities[i]
  ethnicity = reference_ethnicities[i]
  matches = candidate_identities[cos_row>thresh]  
  not_matches = candidate_identities[cos_row<=thresh]
  TP = sum(matches == identity)
  FP = sum(matches != identity)
  TN = sum(not_matches != identity)
  FN = sum(not_matches == identity)
  

  row = {'reference_identity': identity,
           'reference_ethnicity': ethnicity, 
           'TP': TP,
           'TN': TN,
           'FP': FP,
           'FN': FN,
           'matches': matches, 
           'not_matches': not_matches}
  cos_verification = cos_verification.append(row,ignore_index=True)

print(cos_verification[cos_verification.FN == 0][cos_verification.FP == 0])
African_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'African']
print(African_cos.sum(axis=0))
Asian_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'Asian']
print(Asian_cos.sum(axis=0))
Indian_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'Indian']
print(Indian_cos.sum(axis=0))
Caucasian_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'Caucasian']
print(Caucasian_cos.sum(axis=0))

  0%|          | 0/11403 [00:00<?, ?it/s]

      reference_identity reference_ethnicity TP     TN FP FN  \
4              m.0p8s_gx             African  2  29115  0  0   
17             m.047d6rm             African  2  29115  0  0   
32              m.041x_3             African  3  29114  0  0   
38              m.03dwmn             African  1  29116  0  0   
57             m.02qvq8h             African  1  29116  0  0   
...                  ...                 ... ..    ... .. ..   
11374          m.03h654p              Indian  4  29113  0  0   
11379           m.07pkcf              Indian  3  29114  0  0   
11383          m.047t0lb              Indian  3  29114  0  0   
11384           m.0cp4n7              Indian  6  29111  0  0   
11391           m.0gxtsj              Indian  2  29115  0  0   

                                                 matches  \
4                                 [m.0p8s_gx, m.0p8s_gx]   
17                                [m.047d6rm, m.047d6rm]   
32                        [m.041x_3, m.041x_3, m.04

In [12]:
thresh = 0.65
cor_verification = pd.DataFrame(columns=['reference_identity','reference_ethnicity','TP','TN','FP','FN','matches','not_matches'])
for i, cor_row in tqdm_notebook(enumerate(cor),total=len(cor)):
  identity = reference_identities[i]
  ethnicity = reference_ethnicities[i]
  matches = candidate_identities[cor_row>thresh]  
  not_matches = candidate_identities[cor_row<=thresh]
  TP = sum(matches == identity)
  FP = sum(matches != identity)
  TN = sum(not_matches != identity)
  FN = sum(not_matches == identity)
  

  row = {'reference_identity': identity,
           'reference_ethnicity': ethnicity, 
           'TP': TP,
           'TN': TN,
           'FP': FP,
           'FN': FN,
           'matches': matches, 
           'not_matches': not_matches}
  cor_verification = cor_verification.append(row,ignore_index=True)

print(cor_verification[cor_verification.FN == 0][cor_verification.FP == 0])

African_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'African']
print(African_cos.sum(axis=0))
Asian_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'Asian']
print(Asian_cos.sum(axis=0))
Indian_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'Indian']
print(Indian_cos.sum(axis=0))
Caucasian_cos = cos_verification.loc[cos_verification.reference_ethnicity == 'Caucasian']
print(Caucasian_cos.sum(axis=0))

  0%|          | 0/11403 [00:00<?, ?it/s]

      reference_identity reference_ethnicity TP     TN FP FN  \
9               m.0b49ph             African  2  29115  0  0   
38              m.03dwmn             African  1  29116  0  0   
56             m.064qbj5             African  3  29114  0  0   
74              m.04_zbx             African  2  29115  0  0   
76              m.0g5576             African  2  29115  0  0   
...                  ...                 ... ..    ... .. ..   
11299           m.08lj5b              Indian  1  29116  0  0   
11341          m.026n0ps              Indian  1  29116  0  0   
11349           m.0bfm55              Indian  3  29114  0  0   
11353           m.0h19lp              Indian  2  29115  0  0   
11379           m.07pkcf              Indian  3  29114  0  0   

                                 matches  \
9                   [m.0b49ph, m.0b49ph]   
38                            [m.03dwmn]   
56     [m.064qbj5, m.064qbj5, m.064qbj5]   
74                  [m.04_zbx, m.04_zbx]   
76         

In [23]:
len(asian_images.identityID.unique())

2492